In [7]:
import numpy as np
import pandas as pd
import os
import librosa
import soundfile as sf
import json
import math
import pickle
from pathlib import Path
from feature_generator import create_features

In [8]:
# Runtime environment variables

local_Dump = False

modelstore = Path.cwd() / 'models'
if not os.path.exists(modelstore):
    os.makedirs(modelstore)  

In [9]:
# Run this cell if you want to read a model only that exist already


Model_Name = 'MEL_Linear_1'

param_file = Model_Name

Feature_Params = {}

if Path(modelstore / param_file).is_file():

    with open(modelstore / param_file, 'r') as file:
        Feature_Params = json.load(file)
    
else:
    print('No such Model Definition:', Model_Name )

Feature_Params

No such Model Definition: MEL_Linear_1


{}

In [10]:
# Define Parameter Dictionary for Feature Extraction for a new Model, run only for a new model

New_Model_Name = 'MEL_Linear_1'

param_file = New_Model_Name


if Path(modelstore / param_file).is_file():
    print ("Modelname is used already!")
    
else:
    print ("Creating new Model File")

 

    Feature_Params = {}

    # Input DIM for fft = fft_window/2 +1, 3 * sampling_rate / hop_length



    Feature_Params['Description'] = 'Only FFT transformation with 2 channels for real and imaginary part, linear absolute values between 0 and 1'
    Feature_Params['Data'] = '/home/mirko/Downloads/IRMAS_Raw.pkl'
    Feature_Params['Transformation'] = 'MEL_linear'
    Feature_Params['sampling_rate'] = 44100
    Feature_Params['hop_length'] = 1024
    Feature_Params['fft_window'] = 1024
    Feature_Params['no_mel_bin'] = 13
    Feature_Params['loudness_normalization'] = 'linear'
    Feature_Params['fmin'] = 0.0
    Feature_Params['fmax'] = 11025.0
    Feature_Params['length_s'] = 3

    # Frequencies = math.ceil(Feature_Params['fft_window'] / 2 +1)
    Frequencies = Feature_Params['no_mel_bin']
    Frames = math.ceil(Feature_Params['length_s'] * Feature_Params['sampling_rate'] / Feature_Params['hop_length'])
    channels = 1
    Feature_Params['Input_Dim'] = [Frequencies, Frames, channels]

    with open(modelstore / param_file, 'w') as file:
        json.dump(Feature_Params, file)

Creating new Model File


In [11]:
path = '/home/mirko/Downloads/IRMAS_Raw.pkl'
data = pd.read_pickle(path)

In [12]:
feature, label = create_features(data, param_file = 'MEL_Linear_1')

Only FFT transformation with 2 channels for real and imaginary part, linear absolute values between 0 and 1
Hop_length:  1024
Sampling Rate: 44100
Fast Fourier Window: 1024
Number of MEL Bins: 13
Shape of Feature:  [13, 130, 1]
Minimum Frequency:  0.0
Maximum Frequency:  11025.0


In [13]:
print(feature.shape)
print(label.shape)

(6705, 13, 130, 1)
(6705,)


In [ ]:
if local_Dump:

    f_name = Model_Name + '_features.npy' 
    l_name = Model_Name + '_labels.npy' 
    np.save(modelstore / f_name, features)
    np.save(modelstore / l_name, labels)